In [1]:
import os, sys
import json
import pandas as pd

In [2]:
with open('train.json', 'r') as f:
    train = json.load(f)
with open('valid.json', 'r') as f:
    valid = json.load(f)
with open('test.json', 'r') as f:
    test = json.load(f)

In [5]:
with open('maven_corpus.json', 'r') as f:
    maven_corpus = json.load(f)

In [34]:
sum([len(item['desc'].split(' ')) < 5 for item in maven_corpus]), len(maven_corpus)

(168156, 310844)

In [7]:
[item for item in maven_corpus if item['name'] == 'org.jenkins-ci.plugins:mailcommander']

[{'name': 'org.jenkins-ci.plugins:mailcommander',
  'desc': 'This plug-in provides function that read a mail subject as a CLI Command',
  'name_tokens': ['org', 'jenkins', 'ci', 'plugins', 'mailcommander'],
  'desc_tokens': ['plug',
   'provides',
   'function',
   'read',
   'mail',
   'subject',
   'cli',
   'command']}]

In [ ]:
[item for item in train if 'maven:org.jenkins-ci.plugins:mailer' in item['labels']]

In [23]:
[item for item in train if 'maven:org.jenkins-ci.plugins:mailcommander' in item['labels']]

[]

In [9]:
lib_names = set(item['name'] for item in maven_corpus)

In [12]:
len(train), len(valid), len(test)

(1860, 620, 633)

In [25]:
[item for item in lib_names if 'jenkins' in item and 'mail' in item]

['org.jenkins-ci.plugins:mail-watcher-plugin',
 'org.jenkinsci.plugins:emailext-template',
 'org.jenkins-ci.plugins:mailcommander',
 'org.jenkins-ci.plugins:poll-mailbox-trigger-plugin',
 'org.jenkins-ci.plugins:mailer',
 'org.jenkins-ci.plugins:job-direct-mail',
 'org.jenkins-ci.plugins:email-ext',
 'org.jenkins-ci.plugins:mailmap-resolver',
 'org.jenkins-ci.plugins:email-ext-recipients-column',
 'io.jenkins.plugins:validating-email-parameter',
 'org.jenkins-ci.plugins:poll-mailbox-trigger']

In [ ]:
[item['top_k'] for item in test if item['cve_id'] == 'CVE-2020-2318']

In [35]:
with open('test_full_names.json', 'r') as f:
    test_full_names = json.load(f)
with open('test_zero_names.json', 'r') as f:
    test_zero_names = json.load(f)

In [38]:
test_full = [item for item in test if item['cve_id'] in test_full_names]
test_zero = [item for item in test if item['cve_id'] in test_zero_names]

In [39]:
len(test_full), len(test_zero)

(405, 228)

In [41]:
with open('test_full.json', 'w') as f:
    json.dump(test_full, f)
with open('test_zero.json', 'w') as f:
    json.dump(test_zero, f)

In [45]:
310844 * 200000 / 1000 / 3600 /24

719.5462962962962

In [47]:
verajava = pd.read_csv('java_tot_df.csv')

In [48]:
verajava.columns

Index(['Unnamed: 0', 'cve_id', 'labels', 'num'], dtype='object')

In [50]:
nums = list(verajava['num'])

In [51]:
rec = sum([1.0 / item for item in nums]) / len(nums)

In [54]:
rec

0.6195080115349703

In [53]:
rec* 2 / (1 + rec)

0.7650570508111292

In [55]:
cves = list(verajava.cve_id)

In [58]:
vera_new = [item for item in test if item['cve_id'] in cves]

In [140]:
metric_k = 1
def f1(p, r):
    return 2 * p * r / (p + r)

def output(p, r):
    res = '%0.3f & %0.3f & %0.3f' % (p, r, f1(p, r))
    print(res)
    return res

def prec(item):
    labels = item['labels']
    pred = item['top_k'][:metric_k]
    return sum([item['lib_name'] in labels for item in pred]) / metric_k

def rec(item):
    labels = item['labels']
    if len(labels) == 0:
        return 0.0
    pred = item['top_k'][:metric_k]
    return sum([item['lib_name'] in labels for item in pred]) / len(labels)

def prec_truth(item):
    labels = item['labels']
    return len(labels[:metric_k]) / metric_k

def rec_truth(item):
    labels = item['labels']
    if len(labels) == 0:
        return 0.0
    return len(labels[:metric_k]) / len(labels)

In [75]:
prec = sum([prec(item) for item in vera_new]) / len([item for item in vera_new if len(item['labels']) > 0])
rec = sum([rec(item) for item in vera_new]) / len([item for item in vera_new if len(item['labels']) > 0])
prec, rec, f1(prec, rec)

(0.628140703517588, 0.4046123474515435, 0.4921863640908513)

In [141]:
metric_k = 1
p = sum([prec_truth(item) for item in vera_new]) / len([item for item in vera_new if len(item['labels']) > 0])
r = sum([rec_truth(item) for item in vera_new]) / len([item for item in vera_new if len(item['labels']) > 0])
output(p, r)
metric_k = 2
p = sum([prec_truth(item) for item in vera_new]) / len([item for item in vera_new if len(item['labels']) > 0])
r = sum([rec_truth(item) for item in vera_new]) / len([item for item in vera_new if len(item['labels']) > 0])
output(p, r)
metric_k = 3
p = sum([prec_truth(item) for item in vera_new]) / len([item for item in vera_new if len(item['labels']) > 0])
r = sum([rec_truth(item) for item in vera_new]) / len([item for item in vera_new if len(item['labels']) > 0])
output(p, r)

1.000 & 0.675 & 0.806
0.764 & 0.877 & 0.817
0.598 & 0.949 & 0.734


'0.598 & 0.949 & 0.734'

In [79]:
72 * 24 / 310844 * 512

2.8462379843265433

In [ ]:
test[0]

In [142]:
metric_k = 1
p = sum([prec(item) for item in test]) / len([item for item in test if len(item['labels']) > 0])
r = sum([rec(item) for item in test]) / len([item for item in test if len(item['labels']) > 0])
output(p, r)
metric_k = 2
p = sum([prec(item) for item in test]) / len([item for item in test if len(item['labels']) > 0])
r = sum([rec(item) for item in test]) / len([item for item in test if len(item['labels']) > 0])
output(p, r)
metric_k = 3
p = sum([prec(item) for item in test]) / len([item for item in test if len(item['labels']) > 0])
r = sum([rec(item) for item in test]) / len([item for item in test if len(item['labels']) > 0])
output(p, r)

0.216 & 0.162 & 0.185
0.169 & 0.237 & 0.197
0.142 & 0.290 & 0.191


'0.142 & 0.290 & 0.191'

In [143]:
metric_k = 1
p = sum([prec(item) for item in vera_new]) / len([item for item in vera_new if len(item['labels']) > 0])
r = sum([rec(item) for item in vera_new]) / len([item for item in vera_new if len(item['labels']) > 0])
output(p, r)
metric_k = 2
p = sum([prec(item) for item in vera_new]) / len([item for item in vera_new if len(item['labels']) > 0])
r = sum([rec(item) for item in vera_new]) / len([item for item in vera_new if len(item['labels']) > 0])
output(p, r)
metric_k = 3
p = sum([prec(item) for item in vera_new]) / len([item for item in vera_new if len(item['labels']) > 0])
r = sum([rec(item) for item in vera_new]) / len([item for item in vera_new if len(item['labels']) > 0])
output(p, r)

0.146 & 0.073 & 0.098
0.156 & 0.161 & 0.159
0.141 & 0.215 & 0.170


'0.141 & 0.215 & 0.170'

In [116]:
verajava.columns

Index(['Unnamed: 0', 'cve_id', 'labels', 'num'], dtype='object')

In [119]:
eval(verajava.iloc[2, 2])

['jenkins', 'openshift-origin-cartridge-jenkins', 'jenkins-plugin-openshift']

In [120]:
vera_train = [item for item in train if item['cve_id'] in cves]
vera_test = [item for item in test if item['cve_id'] in cves]

In [124]:
vera_train_labels = set()
for item in vera_train:
    vera_train_labels = vera_train_labels | set(item['labels'])

In [125]:
len(vera_train_labels)

690

In [132]:
vera_test_full = [item for item in vera_test if len(set(item['labels']) & vera_train_labels) == len(set(item['labels']))]
vera_test_zero = [item for item in vera_test if len(set(item['labels']) & vera_train_labels) != len(set(item['labels']))]
len(vera_test_full), len(vera_test_zero)

(116, 83)

In [134]:
vera_test_zero = [item for item in vera_test if len(set(item['labels']) & vera_train_labels) == 0]
vera_test_full = [item for item in vera_test if len(set(item['labels']) & vera_train_labels) != 0]
len(vera_test_full), len(vera_test_zero)

(144, 55)

In [137]:
with open('test_zero.json', 'r') as f:
    test_zero = json.load(f)
with open('test_full.json', 'r') as f:
    test_full = json.load(f)

In [139]:
len(test_full), len(test_zero)

(405, 228)

In [144]:
metric_k = 1
p = sum([prec(item) for item in vera_test_zero]) / len([item for item in vera_new if len(item['labels']) > 0])
r = sum([rec(item) for item in vera_test_zero]) / len([item for item in vera_new if len(item['labels']) > 0])
output(p, r)
metric_k = 2
p = sum([prec(item) for item in vera_test_zero]) / len([item for item in vera_new if len(item['labels']) > 0])
r = sum([rec(item) for item in vera_test_zero]) / len([item for item in vera_new if len(item['labels']) > 0])
output(p, r)
metric_k = 3
p = sum([prec(item) for item in vera_test_zero]) / len([item for item in vera_new if len(item['labels']) > 0])
r = sum([rec(item) for item in vera_test_zero]) / len([item for item in vera_new if len(item['labels']) > 0])
output(p, r)

0.050 & 0.037 & 0.043
0.050 & 0.073 & 0.060
0.042 & 0.090 & 0.057


'0.042 & 0.090 & 0.057'

In [145]:
metric_k = 1
p = sum([prec(item) for item in test_zero]) / len([item for item in vera_new if len(item['labels']) > 0])
r = sum([rec(item) for item in test_zero]) / len([item for item in vera_new if len(item['labels']) > 0])
output(p, r)
metric_k = 2
p = sum([prec(item) for item in test_zero]) / len([item for item in vera_new if len(item['labels']) > 0])
r = sum([rec(item) for item in test_zero]) / len([item for item in vera_new if len(item['labels']) > 0])
output(p, r)
metric_k = 3
p = sum([prec(item) for item in test_zero]) / len([item for item in vera_new if len(item['labels']) > 0])
r = sum([rec(item) for item in test_zero]) / len([item for item in vera_new if len(item['labels']) > 0])
output(p, r)

0.246 & 0.222 & 0.233
0.156 & 0.273 & 0.198
0.124 & 0.318 & 0.178


'0.124 & 0.318 & 0.178'

In [146]:
metric_k = 1
p = sum([prec(item) for item in vera_test_full]) / len([item for item in vera_new if len(item['labels']) > 0])
r = sum([rec(item) for item in vera_test_full]) / len([item for item in vera_new if len(item['labels']) > 0])
output(p, r)
metric_k = 2
p = sum([prec(item) for item in vera_test_full]) / len([item for item in vera_new if len(item['labels']) > 0])
r = sum([rec(item) for item in vera_test_full]) / len([item for item in vera_new if len(item['labels']) > 0])
output(p, r)
metric_k = 3
p = sum([prec(item) for item in vera_test_full]) / len([item for item in vera_new if len(item['labels']) > 0])
r = sum([rec(item) for item in vera_test_full]) / len([item for item in vera_new if len(item['labels']) > 0])
output(p, r)

0.095 & 0.036 & 0.052
0.106 & 0.088 & 0.096
0.099 & 0.125 & 0.111


'0.099 & 0.125 & 0.111'

In [147]:
metric_k = 1
p = sum([prec(item) for item in test_full]) / len([item for item in vera_new if len(item['labels']) > 0])
r = sum([rec(item) for item in test_full]) / len([item for item in vera_new if len(item['labels']) > 0])
output(p, r)
metric_k = 2
p = sum([prec(item) for item in test_full]) / len([item for item in vera_new if len(item['labels']) > 0])
r = sum([rec(item) for item in test_full]) / len([item for item in vera_new if len(item['labels']) > 0])
output(p, r)
metric_k = 3
p = sum([prec(item) for item in test_full]) / len([item for item in vera_new if len(item['labels']) > 0])
r = sum([rec(item) for item in test_full]) / len([item for item in vera_new if len(item['labels']) > 0])
output(p, r)

0.367 & 0.238 & 0.289
0.324 & 0.399 & 0.358
0.280 & 0.505 & 0.360


'0.280 & 0.505 & 0.360'